<a href="https://colab.research.google.com/github/yseeker/pytorch-gan/blob/main/cycle_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import argparse
import glob
import os

import torch
import tqdm
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dset
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import torchvision.transforms as transforms

In [2]:
class CFG:
    n_epoch = 200
    batch_size = 1
    lr = 2e-4
    decay_start = 100
    weight_identity = 5.0
    weight_cycle = 10.0
    beta1 = 0.5
    image_size = 256

In [3]:
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    def __init__(selftransform = None):
        super().__init__()
        self.files_A = glob.glob('./drive/MyDrive/data/horse2zebra/trainA/*.jpg')
        self.files_B = glob.glob('./drive/MyDrive/data/horse2zebra/trainB/*.jpg')
        self.transform = transform

    def __getitem__(self, index):
        imgA = self.transform(self.files_A[index % len(self.files_A)])
        while True:
            random_index = np.random/randint(0, len(self.files_B)-1)
            imgB = self.transform(Image.open(self.file_B[random_index % len(self.file_B)]))
            if C == 3:
                break
        return {'A' : imgA, 'B' : imgB}
        
    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

In [ ]:
class DecayLR(object):
    def __init__(sself, n_epochs, offset, decay_start_epoch):
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch
    
    def step(self, epoch):
        return 1.0- max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)

class ReplayBuffer(object):
    def __init__(self, max_size = 50):
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size():
                self.data.append(element)
                to_return.append(element)
            else:
                if np.random.rand() > 0.5:
                    i = np.random.randint(0, self.max_size -1)
                    to_retuna.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return torch.cat(to_return)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels),
            nn.ReLU(inplace = True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels),
            nn.ReLU(inplace = True),
        )

    def forward(self, x):
        out = self.conv_layers(x)
        out += x
        return out

class Generator(nn.Module):
    def __init__(self, res_block, in_channels):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.ReflectionPad2d(3),
            nn.Conv2d(in_channels, 64, 7)
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 3, stride = 2, padding = 1)
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 3, stride = 2, padding = 1)
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
        )

        self.res_block = res_block(256)
        self.transformer = nn.ModuleList(9*[self.res_block])
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride = 2, padding = 1, output_padding = 1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace = True)
            nn.ConvTranspose2d(128, 64, 3, stride = 2, padding = 1, output_padding = 1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace = True)
            nn.ReflectionPad2d(3),
            nn.Conv2d(63, 3, 7),
            nn.Tanh()
        )

    def forward(self, x):
        out = self.encoder(x)
        for func in self.transformer:
            out = func(out)
        out = self.decoder(out)
        return out


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = self.conv_layer(3, 64, 4, 2, 1, False)
        self.conv2 = self.conv_layer(64, 128, 4, 2, 1, False)
        self.conv3 = self.conv_layer(128, 256, 4, 2, 1, False)
        self.conv4 = self.conv_layer(256, 512, 4, 1, 1, False)
        self.conv5 = nn.Conv2d(512, 1, 4, 1, padding = 1)

    @staticmethod
    def conv_layer(in_channels, out_channels, kernel_size, stride, padding, has_norm = False):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)]
        if has_norm:
            layers.append(nn.InstanceNorm2d(out_channels))
        layeres.append(nn.LeakyReLU(0.2, inplace = True))
        net = nn.Sequential(*layers)
        return net

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        B, C, H, W = out.size()
        out = F.avg_pool2d(out, (H, W))
        out = out.view(B, -1)
        return out

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
import torchvision.transforms as transforms
tranform = transforms.Compose([
    transforms.Resize(int(CFG.image_size * 1.12), Image.BICUBIC),
    transforms.RandomCrop(opt.image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5, ))
])

In [ ]:
dataset = ImageDataset(transforms = transform)
dataloader = DataLoader(dataset = dataset, batch_size = CFG.batch_size, shuffle = True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

netG_A2B = Generator(ResidualBlock).to(device)
netG_B2A = Generator(ResidualBlock).to(device)
netD_A = Discriminator().to(device)
netD_B = Discriminator().to(device)

netD_A.apply(weights_init)
netD_B.apply(weights_init)


discriminator = Discrimnator(nc = 1, ndf = CFG.nch_d).to(device)
netG_r.apply(weights_init)

criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr = CFG.lr, betas=(CFG.beta1, 0.999), weight_decay = 1e-5)
optimizer_D = optim.Adam(discriminator.parameters(), lr = CFG.lr, betas=(CFG.beta1, 0.999), weight_decay = 1e-5)

model_name = 'DCGGAN'
f_path_result = './drive/MyDrive/result/{}'.format(model_name)
f_path_params = './drive/MyDrive/result/{}'.format(model_name)

os.makedirs(f_path_result, exist_ok = True)
os.makedirs(f_path_params, exist_ok = True)

In [ ]:
adversarial_loss = torch.nn.MSELoss().to(device)
cycle_loss = torch.nn.L1Loss().to(device)
identity_loss = torch.nn.L1Loss().to(device)

optimizer_D_A = optim.Adam(netD_A.parameters(), lr = CFG.lr, betas=(CFG.beta1, 0.999), weight_decay = 1e-5)
optimizer_D_B = optim.Adam(netD_B.parameters(), lr = CFG.lr, betas=(CFG.beta1, 0.999), weight_decay = 1e-5)
optimizer_G = optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()), lr = CFG.lr, betas = (CFG.beta1, 0.999))

In [4]:
lr_lambda = DecayLR(CFG.n_epochs, 0, CFG.decay_start).step
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=lr_lambda)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=lr_lambda)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=lr_lambda)

import os
model_name = 'CycleGAN'
f_path_result = './drive/MyDrive/result/{}'.format(model_name)
f_path_params = './drive/MyDrive/params/{}'.format(model_name)

os.makedirs(f_path_result, exist_ok = True)
os.makedirs(f_path_params, exist_ok = True)

NameError: ignored

In [ ]:
def save_params(epoch, dir_path, model_list, model_name_list):
    for model, model_name in zip(model, model_name_list):
        file_path = dir_path + '/{model}_{epoch}.pth'.format(model = model_name, epoch = epoch)
        torch.save(model.state_dict(), file_path)

In [5]:
fake_A_Buffer = ReplayBuffer()
fake_B_Buffer = ReplayBuffer()

for epoch in range(0, CFG.n_epoch):
    running_loss_D = 0.0
    running_loss_G = 0.0
    running_loss_G_GAN = 0.0
    running_loss_G_cycle = 0.0
    running_loss_G_identity = 0.0
    for data in tqdm.tqdm(dataloader):
        real_img_A = data['A'].to(device)
        real_img_B = data['B'].to(device)
        batch_size = real_img_A.size()[0]
        real_label = torch.ones([batch_size, 1]).to(device)
        fake_label = torch.zeros([batch_size, 1]).to(device)

        loss_GAN_A2B = adversarial_loss(output_B, real_label)
        loss_GAN_B2A = adversarial_loss(output_A, real_label)

        cycle_img_A = netG_B2A(fake_img_B)
        cycle_img_B = netG_A2B(fake_img_A)

        loss_cycle_ABA = cycle_loss(cycle_img_A, real_img_A)
        loss_cycle_BAB = cycle_loss(cycle_img_B, real_img_B)

        identity_img_A = netG_B2A(real_img_A)
        identity_img_B = netG_A2B(real_img_B)
        loss_identity_A = identity_loss(identity_img_A, real_img_A)
        loss_identity_B = identity_loss(identity_img_B, real_img_B)

        lossG = (loss_GAN_A2B + loss_GAN_B2A 
                 + CFG.weight_identity*(loss_identity_A + loss_identity_B) 
                 + CFG.weight_cycle * (loss_cycle_ABA + loss_cycle_BAB))
        lossG.backward()
        optimizer_G.step()

        optimizer_D_A.zero_grad()
        optimizer_D_B.zero_grad()

        real_output_A = netD_A(real_img_A)
        real_output_B = netD_B(real_img_B)
        loss_DA_real = adversarial_loss(real_output_A, real_label)
        loss_DB_real = adversarial_loss(real_output_B, real_label)

        fake_img_A = fake_A_buffer.push_and_pop(fake_img_A)
        fake_img_B = fake_B_buffer.push_and_pop(fake_img_B)
        fake_output_A = netD_A(fake_img_A.detach())
        fake_output_B = netD_B(fake_img_B.detach())

        loss_DA_fake = adversarial_loss(fake_output_A, fake_label)
        loss_DB_fake = adversarial_loss(fake_output_B, fake_label)

        loss_DA = (loss_DA_real + loss_DA_fake)*0.5
        loss_DB = (loss_DB_real + loss_DB_fake)*0.5

        loss_DA.backward()
        loss_DB.backward()
        optimizer_D_A.step()
        optimizer_D_B.step()

        running_loss_D += (loss_DA.item() + loss_DB.item()) / 2.0
        running_loss_G += lossG.item()
        running_loss_G_GAN += (loss_GAN_A2B.item() + loss_GAN_B2A.item()) / 2.0
        running_loss_G_cycle += (loss_cycle_ABA.item() + loss__BAB.item()) / 2.0
        running_loss_G_identity += (loss_identity_A.item() + loss_identity_B.item()) / 2.0

    lr_schedular_G.step()
    lr_schedular_D_A.step()
    lr_schedular_D_B.step()
    running_loss_D /= len(dataloader)
    running_loss_G /= len(dataloader)
    running_loss_G_GAN /= len(dataloader)
    running_loss_G_cycle /= len(dataloader)
    running_loss_G_identity /= len(dataloader)
    loss_log = "epoch: {}, Loss D:"




NameError: ignored

In [ ]:
def train_one_step(real_imgs, noise, labels_valid, labels_fake):
    """Train Discriminator"""
    optimizer_D.zero_grad()
    gen_imgs = generator(noise)
    out_real = discriminator(real_imgs)
    out_fake = discriminator(gen_imgs.detach())
    real_loss = criterion(out_real, labels_valid)
    fake_loss = criterion(out_fake, labels_fake)
    d_loss = real_loss + fake_loss
    d_loss.backward()
    optimizer_D.step()

    """Train Generator"""
    optimizer_G.zero_grad()
    # Loss measures generator's ability to fool the discriminator
    g_loss = criterion(discriminator(gen_imgs), labels_valid)
    g_loss.backward()
    optimizer_G.step()

    return gen_imgs, g_loss, d_loss

In [ ]:
lossesD = []
lossesG = []
raw_lossesD = []
raw_lossesG = []
for epoch in range(CFG.n_epoch):
    running_lossD = 0.0
    running_lossG = 0.0
    for i, (real_imgs, labels) in enumerate(tqdm.tqdm(dataloader, position = 0)):
        batch_size = real_imgs.size()[0]
        shape = (batch_size, 1, 1, 1)
        real_imgs = real_imgs.to(device)
        noise = torch.randn(batch_size, CFG.z_dim, 1, 1).to(device)
        labels_real = torch.ones(shape).to(device)
        labels_fake = torch.zeros(shape).to(device)

        fake_imgs, lossG, lossD = train_one_step(real_imgs, noise, labels_real, labels_fake)

        running_lossD += lossD.item()
        running_lossG += lossG.item()
        raw_lossesD.append(lossD.item())
    running_lossD /= len(dataloader)
    running_lossG /= len(dataloader)
    print('epoch: {}, lossD: {}, lossG: {}'.format(epoch, running_lossD, running_lossG))
    lossesD.append(running_lossD)
    lossesG.append(running_lossG)

    # show fake imgs
    grid_imgs = vutils.make_grid(fake_imgs[:24].detach())
    grid_imgs_arr = grid_imgs.cpu().numpy()
    plt.imshow(np.transpose(grid_imgs_arr, (1,2,0)))
    plt.show()

    # save imgs and parameters
    vutils.save_image(fake_imgs, f_path_result + '/{}.jpg'.format(epoch))
    save_params(f_path_params, epoch, discriminator, generator)